In [ ]:
!pip install redis numpy pandas
!pip install -U sentence-transformers
!pip install openai
!pip install langchain

In [ ]:
import numpy as np
import pandas as pd
import time
from redis.commands.search.field import VectorField
from redis.commands.search.field import TextField
from redis.commands.search.field import TagField
from redis.commands.search.query import Query
import redis
from sentence_transformers import SentenceTransformer

MAX_TEXT_LENGTH=512
NUMBER_PRODUCTS=10000

def auto_truncate(val):
    return val[:MAX_TEXT_LENGTH]

all_prods_df = pd.read_csv("/content/drive/MyDrive/Practicals & Contests/flipG/flip_products.csv", converters={'description': auto_truncate})
all_prods_df['primary_key'] = all_prods_df['_id']
all_prods_df.drop(columns = ['out_of_stock', 'images','discount', 'average_rating', 'actual_price','crawled_at','seller','selling_price'], inplace=True, axis=1)
all_prods_df['description'].replace('', np.nan, inplace=True)
# all_prods_df['discount'].replace('', np.nan, inplace=True)
# all_prods_df['actual_price'].replace('', np.nan, inplace=True)
# all_prods_df['average_rating'].replace('', np.nan, inplace=True)
all_prods_df['brand'].replace('', np.nan, inplace=True)
# all_prods_df['seller'].replace('', np.nan, inplace=True)
# all_prods_df['selling_price'].replace('', np.nan, inplace=True)

all_prods_df['vectoring_attribute'] = "Title: " + all_prods_df['title'] + "Category: " + all_prods_df['sub_category'] + "Details: " + all_prods_df['product_details']  + "Additional Desc: " + all_prods_df['description']

all_prods_df.dropna(subset=['vectoring_attribute'], inplace=True)
all_prods_df.reset_index(drop=True,inplace=True)

product_metadata = all_prods_df.to_dict(orient='index')

all_prods_df.head()

,_id,brand,category,description,pid,product_details,sub_category,title,url,primary_key,vectoring_attribute
0,fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a,York,Clothing and Accessories,Yorker trackpants made from 100% rich combed c...,TKPFCZ9EA7H5FYZH,"[{'Style Code': '1005COMBO2'}, {'Closure': 'El...",Bottomwear,Solid Men Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...,fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a,Title: Solid Men Multicolor Track PantsCategor...
1,893e6980-f2a0-531f-b056-34dd63fe912c,York,Clothing and Accessories,Yorker trackpants made from 100% rich combed c...,TKPFCZ9EJZV2UVRZ,"[{'Style Code': '1005BLUE'}, {'Closure': 'Draw...",Bottomwear,Solid Men Blue Track Pants,https://www.flipkart.com/yorker-solid-men-blue...,893e6980-f2a0-531f-b056-34dd63fe912c,Title: Solid Men Blue Track PantsCategory: Bot...
2,eb4c8eab-8206-59d0-bcd1-a724d96bf74f,York,Clothing and Accessories,Yorker trackpants made from 100% rich combed c...,TKPFCZ9EHFCY5Z4Y,"[{'Style Code': '1005COMBO4'}, {'Closure': 'El...",Bottomwear,Solid Men Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...,eb4c8eab-8206-59d0-bcd1-a724d96bf74f,Title: Solid Men Multicolor Track PantsCategor...
3,3f3f97bb-5faf-57df-a9ff-1af24e2b1045,York,Clothing and Accessories,Yorker trackpants made from 100% rich combed c...,TKPFCZ9ESZZ7YWEF,"[{'Style Code': '1005COMBO3'}, {'Closure': 'El...",Bottomwear,Solid Men Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...,3f3f97bb-5faf-57df-a9ff-1af24e2b1045,Title: Solid Men Multicolor Track PantsCategor...
4,750caa3d-6264-53ca-8ce1-94118a1d8951,York,Clothing and Accessories,Yorker trackpants made from 100% rich combed c...,TKPFCZ9EVXKBSUD7,"[{'Style Code': '1005COMBO1'}, {'Closure': 'Dr...",Bottomwear,"Solid Men Brown, Grey Track Pants",https://www.flipkart.com/yorker-solid-men-brow...,750caa3d-6264-53ca-8ce1-94118a1d8951,"Title: Solid Men Brown, Grey Track PantsCatego..."


In [ ]:
# run-the-way-preprocessing

MAX_TEXT_LENGTH=512
NUMBER_PRODUCTS=10000

def auto_truncate(val):
    return val[:MAX_TEXT_LENGTH]

all_prods_df = pd.read_csv("/content/drive/MyDrive/Practicals & Contests/flipG/rtr_fin1.csv", converters={'review_text': auto_truncate})
all_prods_df['primary_key'] = str(" ")
all_prods_df['vectoring_attribute'] = str(" ")
for i in range(len(all_prods_df)):
  all_prods_df['primary_key'][i] = str(all_prods_df['user_id'][i]) + '-' + str(all_prods_df['item_id'][i])
all_prods_df.drop(columns = ['review_date', 'review_summary','rating'], inplace=True, axis=1)
all_prods_df['review_text'].replace('', np.nan, inplace=True)
# all_prods_df['discount'].replace('', np.nan, inplace=True)
# all_prods_df['actual_price'].replace('', np.nan, inplace=True)
# all_prods_df['average_rating'].replace('', np.nan, inplace=True)
# all_prods_df['brand'].replace('', np.nan, inplace=True)
# all_prods_df['seller'].replace('', np.nan, inplace=True)
# all_prods_df['selling_price'].replace('', np.nan, inplace=True)

for i in range(len(all_prods_df)):
  all_prods_df['vectoring_attribute'][i] = "Product category " + str(all_prods_df['category'][i]) + " #Body-type " + str(all_prods_df['body type'][i]) +  " #Product is " +str(all_prods_df['fit'][i])+ ' #Worn for an event or ocassion of '+str(all_prods_df['rented for'][i])+ " #I am of age " + str(all_prods_df['age'][i]) + " years old. Height " + str(all_prods_df['height'][i]) + ' n Weight ' + str(all_prods_df['weight'][i]) + " #My bust size " + str(all_prods_df['bust size'][i]) +" n my size "+ str(all_prods_df['size'][i]) + " #Desc " + str(all_prods_df['review_text'][i])

all_prods_df.dropna(subset=['vectoring_attribute'], inplace=True)
all_prods_df.reset_index(drop=True,inplace=True)

product_metadata = all_prods_df.to_dict(orient='index')

all_prods_df.head()

<ipython-input-13-5a1adff7768e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_prods_df['primary_key'][i] = str(all_prods_df['user_id'][i]) + '-' + str(all_prods_df['item_id'][i])
<ipython-input-13-5a1adff7768e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_prods_df['vectoring_attribute'][i] = "Product category " + str(all_prods_df['category'][i]) + " #Body-type " + str(all_prods_df['body type'][i]) +  " #Product is " +str(all_prods_df['fit'][i])+ ' #Worn for an event or ocassion of '+str(all_prods_df['rented for'][i])+ " #I am of age " + str(all_prods_df['age'][i]) + " years old. Height " + str(all_prod

,fit,user_id,bust size,item_id,weight,rented for,review_text,body type,category,height,size,age,primary_key,vectoring_attribute
0,fit,420272,34d,2260466,137lbs,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,romper,"5' 8""",14.0,28.0,420272-2260466,Product category romper #Body-type hourglass #...
1,fit,273551,34b,153475,132lbs,other,I rented this dress for a photo shoot. The the...,straight & narrow,gown,"5' 6""",12.0,36.0,273551-153475,Product category gown #Body-type straight & na...
2,fit,360448,NaN,1063761,NaN,party,This hugged in all the right places! It was a ...,NaN,sheath,"5' 4""",4.0,116.0,360448-1063761,Product category sheath #Body-type nan #Produc...
3,fit,909926,34c,126335,135lbs,formal affair,I rented this for my company's black tie award...,pear,dress,"5' 5""",8.0,34.0,909926-126335,Product category dress #Body-type pear #Produc...
4,fit,151944,34b,616682,145lbs,wedding,I have always been petite in my upper body and...,athletic,gown,"5' 9""",12.0,27.0,151944-616682,Product category gown #Body-type athletic #Pro...


In [ ]:
print(len(product_metadata))
# Check one of the products
product_metadata[0]

3180


{'fit': 'fit',
 'user_id': 420272,
 'bust size': '34d',
 'item_id': 2260466,
 'weight': '137lbs',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14.0,
 'age': 28.0,
 'primary_key': '420272-2260466',
 'vectoring_attribute': 'Product category romper #Body-type hourglass #Product is fit #Worn for an event or ocassion of vacation #I am of age 28.0 years old. Height 5\' 8" n Weight 137lbs #My bust size 34d n my size 14.0 #Desc An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that\'s to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.'}

In [ ]:
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

In [ ]:
vectoring_attribute =  [product_metadata[i]['vectoring_attribute']  for i in product_metadata.keys()]
print(len(vectoring_attribute),"    " , len(product_metadata))

3180      3180


In [ ]:
vectoring_attribute_vectors = [ model.encode(sentence) for sentence in vectoring_attribute]

print(len(vectoring_attribute_vectors),"    " , len(product_metadata))
# Check one of the products
print(product_metadata[0])
# print(vectoring_attribute_vectors)

3180      3180
{'fit': 'fit', 'user_id': 420272, 'bust size': '34d', 'item_id': 2260466, 'weight': '137lbs', 'rented for': 'vacation', 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.", 'body type': 'hourglass', 'category': 'romper', 'height': '5\' 8"', 'size': 14.0, 'age': 28.0, 'primary_key': '420272-2260466', 'vectoring_attribute': 'Product category romper #Body-type hourglass #Product is fit #Worn for an event or ocassion of vacation #I am of age 28.0 years old. Height 5\' 8" n Weight 137lbs #My bust size 34d n my size 14.0 #Desc An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that\'s to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.'}


In [ ]:
redis_conn = redis.Redis(
  host='redis-17369.c12.us-east-1-4.ec2.cloud.redislabs.com',
  port=17369,
  password='9faloTMw7j7jE95NUOoGOygJIiOIzq8b')

In [ ]:
def load_vectors(client, product_metadata, vector_dict, vector_field_name):
    p = client.pipeline(transaction=False)
    for index in product_metadata.keys():
        #hash key
        key=product_metadata[index]['pid']

        #hash values
        item_metadata = product_metadata[index]
        item_keywords_vector = vector_dict[index].astype(np.float32).tobytes()
        item_metadata[vector_field_name]=item_keywords_vector

        # HSET
        p.hset(key,mapping=item_metadata)

    p.execute()

def create_flat_index (redis_conn,vector_field_name,number_of_vectors, vector_dimensions=512, distance_metric='L2'):
    redis_conn.ft().create_index([
        VectorField(vector_field_name, "FLAT", {"TYPE": "FLOAT32", "DIM": vector_dimensions, "DISTANCE_METRIC": distance_metric, "INITIAL_CAP": number_of_vectors, "BLOCK_SIZE":number_of_vectors }),
        TagField("sub_category"),
        TagField("brand")
    ])

In [ ]:
ITEM_KEYWORD_EMBEDDING_FIELD='vectoring_attribute_vector'
TEXT_EMBEDDING_DIMENSION=768
NUMBER_PRODUCTS=1000

print ('Loading and Indexing + ' +  str(NUMBER_PRODUCTS) + ' products')

#flush all data
redis_conn.flushall()

#create flat index & load vectors
create_flat_index(redis_conn, ITEM_KEYWORD_EMBEDDING_FIELD,NUMBER_PRODUCTS,TEXT_EMBEDDING_DIMENSION,'COSINE')
load_vectors(redis_conn,product_metadata,vectoring_attribute_vectors,ITEM_KEYWORD_EMBEDDING_FIELD)

Loading and Indexing + 1000 products


OutOfMemoryError: ignored

In [ ]:
len(vectoring_attribute_vectors)
len(product_metadata)
# vector_df[0]['pid']

3180

In [ ]:
vector_df = product_metadata
vect_data = []
for i in range(len(vector_df)):
  vect_data.append( (vector_df[i]['pid'], (vectoring_attribute_vectors[i].tolist())) )
vect_data[0]

In [ ]:
vector_df_rtr = product_metadata
vect_data_rtr = []
for i in range(len(vector_df_rtr)):
  vect_data_rtr.append( (vector_df_rtr[i]['primary_key'], (vectoring_attribute_vectors[i].tolist())) )
vect_data_rtr[0]

('420272-2260466',
 [-0.023222338408231735,
  -0.0619061179459095,
  -0.04847130924463272,
  -0.016343604773283005,
  -0.00033216745941899717,
  0.033143673092126846,
  -0.006543040741235018,
  0.001662788912653923,
  0.0545182079076767,
  0.0041461605578660965,
  -0.026794398203492165,
  -0.05655708909034729,
  -0.011162251234054565,
  0.03347715735435486,
  -0.013472240418195724,
  0.05088936537504196,
  0.004766156431287527,
  0.04623918980360031,
  0.04289543256163597,
  -0.12638278305530548,
  0.02623928152024746,
  -0.007302125450223684,
  -0.004513800144195557,
  -0.01113126240670681,
  0.06254459917545319,
  -0.05146118253469467,
  0.0381314717233181,
  0.03285009786486626,
  -0.057872526347637177,
  0.036609768867492676,
  0.03922885283827782,
  -0.028306609019637108,
  0.018979448825120926,
  0.018283966928720474,
  -0.02825196646153927,
  -0.0021276206243783236,
  -0.04565933719277382,
  0.08011734485626221,
  -0.0161133985966444,
  0.0030119989532977343,
  0.030208177864551

In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 10.1 MB/s eta 0:00:00


In [ ]:
import pinecone

#men
# pinecone.init(api_key="8e926ba4-7307-4449-b8d8-12bf61e000cf", environment="us-central1-gcp")

# women
pinecone.init(api_key="", environment="gcp-starter")

In [ ]:
# pinecone.create_index("hax_vectors", dimension=8, metric="euclidean")

# men - grid-hax
# index = pinecone.Index("grid-hax")

# women - grid-runway
index = pinecone.Index("grid-runway")

In [ ]:
i = 500
while(i<3000):
  index.upsert(vect_data_rtr[i : i+500])
  i = i+500
index.upsert(vect_data_rtr[3000:])

{'upserted_count': 180}

In [ ]:
input = " Suggest purse for wedding"
# vectoring_attribute =  [product_metadata[i]['vectoring_attribute']  for i in product_metadata.keys()]
qa_vector = model.encode(input).tolist()
k = 10
results = index.query(
  vector=qa_vector,
  top_k=k,
  include_values=True
)
for i in range(k):
  print(results['matches'][i]['id'])

908171-174086
474307-174086
834172-1547971
475724-935316
965253-130259
318771-452942
694337-1092231
859847-1490515
538528-672419
384189-1793377


In [ ]:
ls = []
for i in range(k):
  val = str( results['matches'][i]['id'] )
  for obj in range(len(product_metadata)):
    if product_metadata[obj]['primary_key'] == val:
        # print(product_metadata[obj]['fit'],product_metadata[obj]['category'], product_metadata[obj]['rented for'], product_metadata[obj]['body type'])
        print(product_metadata[obj]['vectoring_attribute'])

Product category dress #Body-type straight & narrow #Product is fit #Worn for an event or ocassion of wedding #I am of age 29.0 years old. Height 5' 6" n Weight 130lbs #My bust size 34c n my size 1.0 #Desc I rented this dress in both a 0 and a 2 and ended up going with the 0, and it was still a little bit big for me around the waist. It would have been perfect to cinch with a belt, but I couldn't find one in such short notice.Probably the most comfortable dress I've ever worn. The lacy detail around the sleeves were my favorite part, and the pockets were also very helpful while dancing at the wedding. I would highly recommend this dress if you want to be comfortable, and not worry about it being too short. 
Product category dress #Body-type pear #Product is fit #Worn for an event or ocassion of wedding #I am of age 37.0 years old. Height 5' 4" n Weight 128lbs #My bust size 34b n my size 8.0 #Desc I wore this dress to an outdoor spring wedding with gold shoes and accessories. The cockta

In [ ]:
(all_prods_df.query('primary_key == "538528-672419"'))

,fit,user_id,bust size,item_id,weight,rented for,review_text,body type,category,height,size,age,primary_key,vectoring_attribute
97,fit,538528,36c,672419,140lbs,wedding,I ended up wearing a different option from Ren...,pear,gown,"5' 2""",15.0,36.0,538528-672419,Product category gown #Body-type pear #Product...


# Misc

In [ ]:
print(vectoring_attribute_vectors)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.3, openai_api_key="")
prompt = PromptTemplate(
    input_variables=["description"],
    template="Create comma seperated product keywords to perform a query on a flipkart dataset for this user input: {description}",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

userinput = input("Hey im a E-commerce Chatbot, how can i help you today? ")
print("User:", userinput)
# Run the chain only specifying the input variable.
keywords = chain.run(userinput)


topK=3
#vectorize the query
query_vector = model.encode(keywords).astype(np.float32).tobytes()

#prepare the query
q = Query(f'*=>[KNN {topK} @{ITEM_KEYWORD_EMBEDDING_FIELD} $vec_param AS vector_score]').sort_by('vector_score').paging(0,topK).return_fields('vector_score','subcategory','brand').dialect(2)
params_dict = {"vec_param": query_vector}

#Execute the query
results = redis_conn.ft().search(q, query_params = params_dict)

full_result_string = ''
for product in results.docs:
    full_result_string += product.sub_category + ' ' + product.brand + ' ' + product.pid + "\n\n\n"


from langchain.memory import ConversationBufferMemory

template = """You are a chatbot. Be kind, detailed and nice. Present the given queried search result in a nice way as answer to the user input. dont ask questions back! just take the given context

{chat_history}
Human: {user_msg}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "user_msg"],
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    llm=OpenAI(model_name="gpt-3.5-turbo", temperature=0.8, openai_api_key="sk-...."),
    prompt=prompt,
    verbose=False,
    memory=memory,
)

answer = llm_chain.predict(user_msg= f"{full_result_string} ---\n\n {userinput}")
print("Bot:", answer)
time.sleep(0.5)

while True:
    follow_up = input("Anything else you want to ask about this topic?")
    print("User:", follow_up)
    answer = llm_chain.predict(
        user_msg=follow_up
    )
    print("Bot:", answer)
    time.sleep(0.5)

Hey im a E-commerce Chatbot, how can i help you today? I am looking for track pants for men
User: I am looking for track pants for men


ResponseError: ignored